# Segment, localise and track


Fixing a bug in the code that meant the intracellular mean intensity values weren't accurate due to the downscaling

In [7]:
from macrohet import dataio, tile
from macrohet.notify import send_sms
import numpy as np
from tqdm.auto import tqdm
from cellpose import models
import btrack 
import torch
import os
from skimage.filters import threshold_otsu
import dask.array as da
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
# defining cellpose model to use
model = models.Cellpose(
                        gpu=True, 
                        model_type='cyto', 
                        net_avg=True, 
                        device=torch.device('cuda')
                        )

Using device: cuda

NVIDIA RTX A6000
Memory Usage:
Allocated: 0.0 GB
Cached:    0.1 GB


### Define functions to tidy up main block of code

In [18]:
def segment(frame, model = model, channels = [0,0], diameter = 325, min_size = 2500):
    
    masks, flows, styles, diams = model.eval(frame, 
                                             channels = channels, 
                                             diameter = diameter, 
                                             min_size = min_size, 
                                             )
    return masks

# define features to use for tracking 
features = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "mean_intensity",
#     "intensity_image"
    ]

scale_factor = 1/5.04

def localise(masks, intensity_image, properties=tuple(features), use_weighted_centroid = False):
    
    # localise objs in images
    objects = btrack.utils.segmentation_to_objects(segmentation=masks,
                                                   intensity_image=intensity_image, 
                                                   properties=properties,
                                                   scale=(scale_factor,scale_factor),
                                                   use_weighted_centroid=use_weighted_centroid, 
                                                   )
                                                   
    return objects

# define config fn to use, using a prob_not_assign = 0.1
config_fn = '/home/dayn/analysis/btrack/models/particle_config_pnassign.json'

def track(objects, masks, config_fn, search_radius = 20):

    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure(config_fn)
        # set max search radius
        tracker.max_search_radius = search_radius
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # redefine features so that both channels are included in track measurements
        tracker.features = list(objects[0].properties.keys())
        # append the objects to be tracked
        tracker.append(objects)
        # set the tracking volume
        tracker.volume=((0, masks.shape[-2]*scale_factor), (0, masks.shape[-1]*scale_factor))
        # track them (in interactive mode)
        tracker.track(step_size=25)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # store the tracks
        tracks = tracker.tracks

    return tracks

def otsu_threshold_stack(images):
    """
    Computes Otsu's threshold value and returns a binary segmentation for
    each image in a time series of grayscale images.

    Parameters:
    -----------
    images : ndarray
        A 3D array of shape (n_images, height, width) containing a time series
        of grayscale images.

    Returns:
    --------
    ndarray
        A boolean array of shape (n_images, height, width) containing the
        binary segmentation for each image in the time series.
    """
    segmentations = np.zeros(images.shape, dtype=bool)
    for i, image in tqdm(enumerate(images), 
                         total=len(images), 
                         leave=False, 
                         desc='Otsu segmenting'):
        loaded_image = image.compute().compute()
        threshold = threshold_otsu(loaded_image)
        segmentations[i] = loaded_image > threshold
    return segmentations

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [9]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/'

In [10]:
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [11]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

# Segment, localise and track

In [12]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images')

In [19]:
### iterate over positions
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):
    if (row, column) == (3, 4):
        continue
    # tile images
    images = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, 
                                 column, 
                                 set_plane = 'sum_proj',
                                 ).astype(np.uint16)

    # reload seg
    tracks_fn = os.path.join(base_dir, f'labels/full_localisation/{row,column}_objs_masks_tracks.h5')
    with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
        masks = hdf.segmentation
        
    # get an otsu segmentation of the Mtb channel
    otsu_mtb = otsu_threshold_stack(images[:,1,...])
    # get a manual thresholding of the mtb channel
    manual_mtb_thresh = images[:,1,...] > 480
    
    # reshape intensity image to be gfp, rfp on last axis for regionprops
    intensity_image = np.stack([images[:,0,...], 
                                images[:,1,...], 
                                otsu_mtb, 
                                manual_mtb_thresh], axis = -1)

    # localise objects
    objects = localise(masks, 
                       intensity_image, 
                       )
    
    # filter out small objects
    objects = [o for o in objects if o.properties['area'] > 500]

    # track on upscaled config fn
    tracks = track(objects, masks, config_fn, search_radius = 20)

    # save out 
    with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/relocalisation/thresh_{row, column}.h5'), 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:
#             writer.write_objects(objects)
            writer.write_tracks(tracks)
            writer.write_segmentation(masks)
    print(f"Checking time indices: {row, column, tracks[0]['t']}")
    send_sms(f"Position {row, column} otsu segmentation saved out, {tracks[0]['t']}")

Progress through positions:   0%|          | 0/24 [00:00<?, ?it/s]

[INFO][2023/05/04 11:37:17 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5...
[INFO][2023/05/04 11:37:36 AM] Loading segmentation (75, 6048, 6048)
[INFO][2023/05/04 11:37:36 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5


Otsu segmenting:   0%|          | 0/75 [00:00<?, ?it/s]

[INFO][2023/05/04 11:45:08 AM] Localizing objects from segmentation...
100%|█████████████████████████████████████████████████████████████████████| 75/75 [17:32<00:00, 14.04s/it]
[INFO][2023/05/04 12:02:41 PM] Objects are of type: <class 'dict'>
[INFO][2023/05/04 12:02:41 PM] ...Found 42674 objects in 75 frames.
[INFO][2023/05/04 12:02:41 PM] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/05/04 12:02:41 PM] Starting BayesianTracker session
[INFO][2023/05/04 12:02:41 PM] Loading configuration file: /home/dayn/analysis/btrack/models/particle_config_pnassign.json
[INFO][2023/05/04 12:02:41 PM] Objects are of type: <class 'list'>
[INFO][2023/05/04 12:03:24 PM] Starting tracking... 
[INFO][2023/05/04 12:03:24 PM] Update using: ['VISUAL', 'MOTION']
[INFO][2023/05/04 12:03:24 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2023/05/04 12:03:29 PM]  - Timing (Bayesian updates: 52.42ms, Linking: 3.31ms)
[INFO][2023/05/04 12:03:29 PM]  - Probabilities (Lin

GLPK Integer Optimizer 5.0
12088 rows, 10269 columns, 14494 non-zeros
10269 integer variables, all of which are binary
Preprocessing...
6044 rows, 10269 columns, 14494 non-zeros
10269 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 6044
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
6044 rows, 10269 columns, 14494 non-zeros
*     0: obj =   3.082479260e+04 inf =   0.000e+00 (2366)
Perturbing LP to avoid stalling [1171]...
Removing LP perturbation [2376]...
*  2376: obj =   1.611108537e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  2376: mip =     not found yet >=              -inf        (1; 0)
+  2376: >>>>>   1.611108537e+04 >=   1.611108537e+04   0.0% (1; 0)
+  2376: mip =   1.611108537e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMA

...

[INFO][2023/05/04 07:15:20 PM] Localizing objects from segmentation...
100%|█████████████████████████████████████████████████████████████████████| 75/75 [20:32<00:00, 16.44s/it]
[INFO][2023/05/04 07:35:53 PM] Objects are of type: <class 'dict'>
[INFO][2023/05/04 07:35:53 PM] ...Found 39356 objects in 75 frames.
[INFO][2023/05/04 07:35:53 PM] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/05/04 07:35:53 PM] Starting BayesianTracker session
[INFO][2023/05/04 07:35:53 PM] Loading configuration file: /home/dayn/analysis/btrack/models/particle_config_pnassign.json
[INFO][2023/05/04 07:35:53 PM] Objects are of type: <class 'list'>
[INFO][2023/05/04 07:35:54 PM] Starting tracking... 
[INFO][2023/05/04 07:35:54 PM] Update using: ['VISUAL', 'MOTION']
[INFO][2023/05/04 07:35:54 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2023/05/04 07:36:00 PM]  - Timing (Bayesian updates: 76.89ms, Linking: 3.39ms)
[INFO][2023/05/04 07:36:00 PM]  - Probabilities (Lin

GLPK Integer Optimizer 5.0
14196 rows, 12279 columns, 17460 non-zeros
12279 integer variables, all of which are binary
Preprocessing...
7098 rows, 12279 columns, 17460 non-zeros
12279 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 7098
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
7098 rows, 12279 columns, 17460 non-zeros
*     0: obj =   3.706062752e+04 inf =   0.000e+00 (3137)
Perturbing LP to avoid stalling [1362]...
Removing LP perturbation [3157]...
*  3157: obj =   1.842087930e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  3157: mip =     not found yet >=              -inf        (1; 0)
+  3157: >>>>>   1.842087930e+04 >=   1.842087930e+04   0.0% (1; 0)
+  3157: mip =   1.842087930e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMA

[INFO][2023/05/04 07:36:09 PM] Opening HDF file: /mnt/DATA/macrohet/labels/relocalisation/thresh_(5, 8).h5...
[INFO][2023/05/04 07:36:10 PM] Writing objects/obj_type_1
[INFO][2023/05/04 07:36:10 PM] Writing labels/obj_type_1
[INFO][2023/05/04 07:36:10 PM] Loading objects/obj_type_1 (28012, 5) (28012 filtered: None)
[INFO][2023/05/04 07:36:10 PM] Writing properties/obj_type_1/area (28012,)
[INFO][2023/05/04 07:36:10 PM] Writing properties/obj_type_1/major_axis_length (28012,)
[INFO][2023/05/04 07:36:10 PM] Writing properties/obj_type_1/minor_axis_length (28012,)
[INFO][2023/05/04 07:36:10 PM] Writing properties/obj_type_1/orientation (28012,)
[INFO][2023/05/04 07:36:10 PM] Writing properties/obj_type_1/mean_intensity (28012, 4)
[INFO][2023/05/04 07:36:10 PM] Writing tracks/obj_type_1
[INFO][2023/05/04 07:36:10 PM] Writing dummies/obj_type_1
[INFO][2023/05/04 07:36:10 PM] Writing LBEP/obj_type_1
[INFO][2023/05/04 07:36:10 PM] Writing fates/obj_type_1
[INFO][2023/05/04 07:36:46 PM] Closin

Checking time indices: (5, 8, [0])


ConnectionError: HTTPSConnectionPool(host='api.twilio.com', port=443): Max retries exceeded with url: /2010-04-01/Accounts/AC5c24c3260ed6b1c8289de1897f4e5b39/Messages.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f86ec9fff70>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [7]:
row, column = 3, 5

In [8]:
images = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, 
                                 column, 
                                 set_plane = 'sum_proj',
                                 ).astype(np.uint16)

# reload seg
tracks_fn = os.path.join(base_dir, f'labels/full_localisation/{row,column}_objs_masks_tracks.h5')
with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
    masks = hdf.segmentation

# reshape intensity image to be gfp, rfp on last axis for regionprops
intensity_image = np.stack([images[:,0,...], 
                            images[:,1,...]], axis = -1)

[INFO][2023/05/03 04:29:42 PM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5...
[INFO][2023/05/03 04:29:54 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/05/03 04:29:54 PM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5


In [31]:
objects = localise(masks[0:2], 
                       intensity_image[0:2], 
                       )

[INFO][2023/05/03 04:41:17 pm] Localizing objects from segmentation...
03-May-23 16:41:17 - btrack.io._localization - INFO     - Localizing objects from segmentation...
100%|███████████████████████████████████████████████████████████████████████| 2/2 [00:29<00:00, 14.66s/it]
[INFO][2023/05/03 04:41:47 pm] Objects are of type: <class 'dict'>
03-May-23 16:41:47 - btrack.io.utils - INFO     - Objects are of type: <class 'dict'>
[INFO][2023/05/03 04:41:47 pm] ...Found 1080 objects in 2 frames.
03-May-23 16:41:47 - btrack.io._localization - INFO     - ...Found 1080 objects in 2 frames.


In [33]:
# seperate array for t0 objs
objs = [o for o in objects_t0 if o.t == 0]

# Adding Mtb threshold to objects

In [10]:
features_to_add = {"my_feature": 0.1}

obj = objects[0]
obj.properties = features_to_add

In [11]:
objects[0]

,ID,x,y,z,t,dummy,states,label,area,major_axis_length,minor_axis_length,orientation,mean_intensity,intensity_image,my_feature
0,0,71.709602,16.420708,0.0,0,False,7,5,21045,214.41384,129.839618,0.75574,"(2,) array","(192, 177, 2) array",0.1


In [13]:
import napari

In [15]:
# v = napari.Viewer()
v.add_image(objects[0].properties['intensity_image'], channel_axis=-1, colormap=['green', 'magenta'])

[<Image layer 'Image' at 0x7faa90e00f70>,
 <Image layer 'Image [1]' at 0x7faa99138ca0>]

In [16]:
rfp_glimpse = objects[0].properties['intensity_image'][...,-1]

In [22]:
for obj in objects:
    v.add_image(obj.properties['intensity_image'], channel_axis=-1, colormap=['green', 'magenta'], name = obj.ID)
    if obj.ID == 10:
        break

### obj 0 and obj 1 are negative and positive respectively

In [26]:
objects[0].properties['mean_intensity'][-1]

array([1194.51385127,  344.16593015])

In [27]:
objects[1].properties['mean_intensity'][-1]

array([1148.94431453,  361.85228883])

In [35]:
len(objs)

536

# Apply a thresholding

In [38]:
rfp_t0 = images[0,1]

In [79]:
images.shape

(75, 2, 6048, 6048)

In [80]:
images[:,1,...]

dask.array<getitem, shape=(75, 6048, 6048), dtype=uint16, chunksize=(1, 2016, 2016), chunktype=numpy.ndarray>

In [84]:
otsu_threshold(images[0:2,1,0:100,0:100])

TypeError: '>=' not supported between instances of 'str' and 'int'

In [99]:
import skimage.filters as filters

def otsu_threshold_single_frame(image):
    """
    Applies Otsu's thresholding segmentation to a grayscale image.

    Parameters:
    -----------
    image : ndarray
        A grayscale image.

    Returns:
    --------
    ndarray
        A binary image with the same shape as the input image.
    """
    # Compute Otsu's threshold value
    thresh = filters.threshold_otsu(image)

    # Apply binary thresholding using the threshold value
    binary_image = image > thresh

    return binary_image


In [47]:
image = rfp_t0.compute().compute()

In [49]:
image.shape

(6048, 6048)

In [67]:
help(filters.threshold_otsu)

Help on function threshold_otsu in module skimage.filters.thresholding:

threshold_otsu(image=None, nbins=256, *, hist=None)
    Return threshold value based on Otsu's method.
    
    Either image or hist must be provided. If hist is provided, the actual
    histogram of the image is ignored.
    
    Parameters
    ----------
    image : (N, M[, ..., P]) ndarray, optional
        Grayscale input image.
    nbins : int, optional
        Number of bins used to calculate histogram. This value is ignored for
        integer arrays.
    hist : array, or 2-tuple of arrays, optional
        Histogram from which to determine the threshold, and optionally a
        corresponding array of bin center intensities. If no hist provided,
        this function will compute it from the image.
    
    
    Returns
    -------
    threshold : float
        Upper threshold value. All pixels with an intensity higher than
        this value are assumed to be foreground.
    
    References
    ----------

In [50]:
# image = rfp_t0[0:100, 0:100]

# Compute Otsu's threshold value
thresh = filters.threshold_otsu(image)
print(thresh)
# Apply binary thresholding using the threshold value
binary_image = image > thresh

1260


In [51]:
v.add_image(binary_image)

<Image layer 'binary_image' at 0x7fa99d690190>

In [52]:
v.add_image(rfp_t0)

<Image layer 'rfp_t0' at 0x7fa97e773be0>

In [56]:
v.add_image(rfp_t0>480)

<Image layer 'Image [1]' at 0x7fa976db3eb0>

# Apply otsu and basic thresholding to glimpse - cannot do

In [70]:
v.add_image(rfp_glimpse)

<Image layer 'rfp_glimpse' at 0x7fa9917aaaf0>

In [71]:
image = rfp_glimpse

In [72]:
thresh = filters.threshold_otsu(image)
print(thresh)
# Apply binary thresholding using the threshold value
binary_image = image > thresh

0


In [74]:
v.add_image(image>480)

<Image layer 'Image' at 0x7fa9688c6700>

# Apply image wide

In [ ]:
%%time
otsu_threshold_single_frame(images[0,1,...])

In [96]:
for i, image in tqdm(enumerate(images[:,1,...])):
    print(i)

0it [00:00, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74


In [94]:
segmentations = np.zeros(rfp_t0.shape, dtype=bool)
for i, image in tqdm(enumerate(rfp_t0)):
    threshold = threshold_otsu(rfp_t0)
    segmentations[i] = rfp_t0 > threshold

0it [00:00, ?it/s]

Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7fa97117a040>
Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/tqdm/notebook.py", line 264, in __iter__
    self.disp(bar_style='danger')
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/tqdm/notebook.py", line 187, in display
    pbar.bar_style = bar_style
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/traitlets/traitlets.py", line 729, in __set__
    self.set(obj, value)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/traitlets/traitlets.py", line 718, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/traitlets/traitlets.py", line 1501, in _notify_trait
    self.notify_change(
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/ipywidgets/widgets/widget.py", line 685, in notify_change
    self.send_state(key=na

KeyboardInterrupt: 

In [87]:
# get an otsu segmentation of the Mtb channel
otsu_mtb = otsu_threshold(images[:,1,...])
# get a manual thresholding of the mtb channel
manual_mtb_thresh = images[:,1,...] > 480

# reshape intensity image to be gfp, rfp on last axis for regionprops
intensity_image = np.stack([images[:,0,...], 
                            images[:,1,...], 
                            otsu_mtb, 
                            manual_mtb_thresh], axis = -1)

# localise objects
objects = localise(masks[0:2], 
                   intensity_image[0:2], 
                   )


KeyboardInterrupt

